<a href="https://colab.research.google.com/github/SnehaValia99/Assignment_NLP/blob/master/Class_assignment_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
from nltk import word_tokenize
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import classification_report, roc_auc_score
import string
from string import digits
from nltk.corpus import stopwords
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split as tts
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np


In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [0]:
t = pd.read_csv('https://raw.githubusercontent.com/zfz/twitter_corpus/master/full-corpus.csv', error_bad_lines=False)
twitter = t[t['Sentiment']!='irrelevant']

In [0]:
def clean(tt):
  tt= tt.lower()
  tt = tt.translate(tt.maketrans('','',digits))#removing digits
  tt = tt.translate(tt.maketrans('', '', '!"@#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'))
  return tt

In [14]:
twitter['TweetTokens']= twitter['TweetText'].apply(clean)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
vectorizer = CountVectorizer(lowercase=True, stop_words=set(stopwords.words('english')))
X = vectorizer.fit_transform(twitter['TweetTokens'])

In [25]:
features = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
X_train,X_test,y_train,y_test = tts(X,twitter['Sentiment'],test_size=0.3, random_state=30)
                                    
X_train = pd.DataFrame(X_train.toarray(),columns=vectorizer.get_feature_names())
x_test = pd.DataFrame(X_test.toarray(),columns=vectorizer.get_feature_names())

rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
y_pred = rfc.predict(x_test)
print(classification_report(rfc.predict(x_test),y_test))#default threshold(0.5)

def multiclass_roc_auc_score(y_test, y_pred, average='macro'):
  lb = LabelBinarizer()
  lb.fit(y_test)
  Y_test = lb.transform(y_test)
  y_pred = lb.transform(y_pred)
  return roc_auc_score(Y_test, y_pred, average=average)

              precision    recall  f1-score   support

    negative       0.39      0.69      0.50       100
     neutral       0.93      0.78      0.85       866
    positive       0.26      0.52      0.34        62

    accuracy                           0.76      1028
   macro avg       0.53      0.66      0.56      1028
weighted avg       0.84      0.76      0.78      1028



In [30]:
multiclass_roc_auc_score(y_test, y_pred)

0.6459931494904095

In [0]:
lbe = LabelEncoder()
Y_test = lbe.fit_transform(y_test)

In [33]:
pred_rfc = rfc.predict_proba(x_test)
pred_rfc

array([[0.03      , 0.96      , 0.01      ],
       [0.02      , 0.36      , 0.62      ],
       [0.12      , 0.69      , 0.19      ],
       ...,
       [0.01      , 0.86713572, 0.12286428],
       [0.28      , 0.71766667, 0.00233333],
       [0.        , 0.92187861, 0.07812139]])

In [0]:
pms = {
    "objective" : "multiclass",
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':3,
    'learning_rate':0.01,
    'max_depth': 7,
    'num_leaves': 127,
    'feature_fraction': 0.4,
    'bagging_freq': 10,
    'num_iterations':1000 ,
    'max_bin' : 32}

In [38]:
lbe = LabelEncoder()  
d_train = lgb.Dataset(X_train,lbe.fit_transform(y_train))
clf = lgb.train( pms,d_train, 100)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [0]:
preds = clf.predict(x_test)
best_preds= [np.argmax(line) for line in preds]

In [40]:
np.bincount((lbe.fit_transform(y_train)))

array([ 397, 1605,  394])

In [41]:
print(classification_report(lbe.inverse_transform(best_preds),y_test))

              precision    recall  f1-score   support

    negative       0.25      0.56      0.34        77
     neutral       0.94      0.75      0.83       922
    positive       0.09      0.38      0.14        29

    accuracy                           0.72      1028
   macro avg       0.43      0.56      0.44      1028
weighted avg       0.87      0.72      0.78      1028



In [42]:
np.bincount((best_preds))

array([ 77, 922,  29])

In [43]:
preds = clf.predict(X_train)
best_preds_train= [np.argmax(line) for line in preds]
print(classification_report(lbe.inverse_transform(best_preds_train),y_train))

              precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396



In [46]:
print("Default Threshold for lgbm : \n",classification_report(lbe.inverse_transform(best_preds_train),y_train))
def thresh_1(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.6:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.6 Threshold for lgbm : \n",classification_report(lbe.inverse_transform(thresh_1(preds)),y_train))

pred_rf = rfc.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(rfc.predict(x_test),y_test)))
print("\n 0.6 threshold for rf : \n",classification_report(lbe.inverse_transform(thresh_1(pred_rf)),y_test) )

Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396


 0.6 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.12      0.85      0.21        55
     neutral       0.84      0.82      0.83      1637
    positive       0.59      0.33      0.43       704

    accuracy                           0.68      2396
   macro avg       0.52      0.67      0.49      2396
weighted avg       0.75      0.68      0.70      2396

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.39      0.69      0.50       100
     neutral       0.93      0.78      0

In [47]:
print("Default Threshold for lgbm : \n",classification_report(lbe.inverse_transform(best_preds_train),y_train))
def thresh_2(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.7:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.7 Threshold for lgbm : \n",classification_report(lbe.inverse_transform(thresh_2(preds)),y_train))
      
pred_rf = rfc.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(rfc.predict(x_test),y_test)))
print("\n 0.7 threshold for rf : \n",classification_report(lbe.inverse_transform(thresh_2(pred_rf)),y_test) )

Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396


 0.7 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.07      0.90      0.12        29
     neutral       0.71      0.86      0.78      1327
    positive       0.75      0.28      0.41      1040

    accuracy                           0.61      2396
   macro avg       0.51      0.68      0.44      2396
weighted avg       0.72      0.61      0.61      2396

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.39      0.69      0.50       100
     neutral       0.93      0.78      0

In [48]:
print("Default Threshold for lgbm : \n",classification_report(lbe.inverse_transform(best_preds_train),y_train))
def thresh_3(array):
  a = []
  for i in range(len(array)):
    if max(array[i])>0.8:
      a.append(np.argmax(array[i]))
    else:
      a.append(2)
  return a
print("\n 0.8 Threshold for lgbm : \n",classification_report(lbe.inverse_transform(thresh_3(preds)),y_train))

pred_rf = rfc.predict_proba(x_test)
print("Default Threshold for rf : \n",(classification_report(rfc.predict(x_test),y_test)))
print("\n 0.8 threshold for rf : \n",classification_report(lbe.inverse_transform(thresh_3(pred_rf)),y_test) )

Default Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.32      0.73      0.44       172
     neutral       0.97      0.74      0.84      2101
    positive       0.25      0.80      0.38       123

    accuracy                           0.74      2396
   macro avg       0.51      0.76      0.55      2396
weighted avg       0.88      0.74      0.79      2396


 0.8 Threshold for lgbm : 
               precision    recall  f1-score   support

    negative       0.03      0.85      0.05        13
     neutral       0.42      0.92      0.58       737
    positive       0.92      0.22      0.35      1646

    accuracy                           0.44      2396
   macro avg       0.46      0.66      0.33      2396
weighted avg       0.76      0.44      0.42      2396

Default Threshold for rf : 
               precision    recall  f1-score   support

    negative       0.39      0.69      0.50       100
     neutral       0.93      0.78      0